# 2 - Calculate Endpoints
In this notebook we calculate the trajectory endpoints from the VED data.

In [1]:
import numpy as np
import pandas as pd
import os
import math
import folium
import networkx as nx

from sklearn.cluster import DBSCAN
from collections import Counter

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
def get_ends(df):
    return df.head(n=1).append(df.tail(n=1))

The `cluster_ends` function applies the DBSCAN clustering algorithm to the extracted trajectory endpoints. The input is a DataFrame with each trajectory's endpoints (start point and end point) only. The output is the same DataFrame with an added column named `end_id` containing the corresponding cluster identifier, or `-1` for a noise point. The data can later be joined to the original DataFrame and thus mark the start and end of each trajectory.

In [3]:
def cluster_ends(df):
    pts = np.radians(df[['Latitude[deg]', 'Longitude[deg]']])

    # Parameters
    eps_in_meters = 50.0
    num_samples = 10

    # Cluster the data
    earth_perimeter = 40070000.0  # In meters
    eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)

    end_clusters = DBSCAN(eps=eps_in_radians, 
                          min_samples=num_samples,
                          metric='haversine',
                          algorithm='ball_tree').fit_predict(pts)
    df['end_id'] = end_clusters
    return df

The `make_path_map` function creates a Folium map based on a list of DataFrames, each with its own trajectory data. This function serves the `make_link_map` function (see below).

In [4]:
def make_path_map(df_list):
    m = folium.Map()
    
    all_points = []
    for df in df_list:
        points = df[['Latitude[deg]', 'Longitude[deg]']].to_numpy()
        polyline = folium.vector_layers.PolyLine(locations=points)
        polyline.add_to(m)
        all_points.extend(points.tolist())
    m.fit_bounds(all_points)
    return m

The `make_link_map` function creates a Folium map based on a _link_, defined as the two terminal node identifiers.

In [5]:
def make_link_map(node_ini, node_end):
    link_df = trip_df[(trip_df.NodeIni == node_ini) & (trip_df.NodeEnd == node_end)]
    
    path_list = []
    for row in link_df.itertuples(index=False):
        path = df[(df.DayNum == row.DayNum) & (df.VehId == row.VehId)].sort_values(by=['DayNum', 'Timestamp(ms)'])
        path_list.append(path)
    return make_path_map(path_list)

## Read the Data
We prepared the dataset file in the previous notebook, so it is readily available for use. Here, we set up a few variables related to the data folder name and file names.

In [6]:
data_path = "./data"
endpoints_file = os.path.join(data_path, "endpoints.parquet")
parquet_file = os.path.join(data_path, "ved.parquet")

Read the dataset from the parquet file. If this file does not exist, please create it by using the code in the previous notebook: `1-convert-ved.ipynb`

In [7]:
df = pd.read_parquet(parquet_file)

In [8]:
df.head(10)

,DayNum,VehId,Trip,Timestamp(ms),Latitude[deg],Longitude[deg],Vehicle Speed[km/h]
10495643,9.558109,2,685,0,42.302569,-83.704196,27.0
10495644,9.558109,2,685,700,42.302569,-83.704196,27.0
10495645,9.558109,2,685,1000,42.302569,-83.704196,30.0
10495646,9.558109,2,685,2500,42.302569,-83.704196,33.0
10495647,9.558109,2,685,2900,42.302580,-83.704604,33.0
10495648,9.558109,2,685,3900,42.302580,-83.704604,38.0
10495649,9.558109,2,685,4900,42.302580,-83.704604,39.0
10495650,9.558109,2,685,5900,42.302580,-83.704604,38.0
10495651,9.558109,2,685,7000,42.302558,-83.705129,37.0
10495652,9.558109,2,685,8000,42.302558,-83.705129,37.0


In [9]:
df[df['Vehicle Speed[km/h]'] == 0].shape

(2574213, 7)

### The Trip DataFrame
The `DayNum` and `VehId` column values uniquely identify individual trips. This means that we can retrieve the data for each trip by grouping by both columns and sorting by the timestamp. The result represents the trip trajectory and it is the source of data for an individual observation for the trajectory clustering process.

In [10]:
trip_df = df.groupby(by=['DayNum', 'VehId']).size().reset_index().rename(columns={0:'Count'})

In [11]:
trip_df['NodeIni'] = -1
trip_df['NodeEnd'] = -1

Here's how the `trip_df` table looks right now.

In [12]:
trip_df.head(10)

,DayNum,VehId,Count,NodeIni,NodeEnd
0,1.002938,550,131,-1,-1
1,1.015493,540,545,-1,-1
2,1.017633,156,690,-1,-1
3,1.025782,588,1150,-1,-1
4,1.054483,267,495,-1,-1
5,1.058991,11,396,-1,-1
6,1.062756,130,452,-1,-1
7,1.065486,174,273,-1,-1
8,1.082547,374,471,-1,-1
9,1.101627,156,697,-1,-1


Each row contains a single trip, but still misses the endpoint node identifiers (`NodeIni` and `NodeEnd`). We are now going to fill in those values so that we can map from any pair of endpoint nodes to the underlying set of trajectories.

In [13]:
def get_trip_ini(row):
    traj = ends_df[(ends_df.DayNum == row.DayNum) & (ends_df.VehId == row.VehId)].head(n=1)
    return traj.end_id.values[0]

def get_trip_end(row):
    traj = ends_df[(ends_df.DayNum == row.DayNum) & (ends_df.VehId == row.VehId)].tail(n=1)
    return traj.end_id.values[0]

## Calculate the Endpoints DataFrame

We start by grouping the data by `DayNum` and `VehId`, thereby identifying unique trajectories.

In [14]:
grp = df.groupby(by=['DayNum', 'VehId'])

Now, we create the endpoints DataFrame, iterating the `get_ends` function through all the grouped data. Each call to the function takes on trip of data.

In [15]:
ends_df = pd.concat([get_ends(d) for k, d in grp])

With all the endpoints identified, we now run the clustering algorithm and store the resulting cluster identifiers in the `end_id` column.

In [16]:
ends_df = cluster_ends(ends_df)

Let's have a look at the data. As you can see, each trip is represented by consecutive row pairs. This arrangement makes it very easy to later group all endpoint cluster identifiers into a list of pairs.

In [17]:
ends_df.head(10)

,DayNum,VehId,Trip,Timestamp(ms),Latitude[deg],Longitude[deg],Vehicle Speed[km/h],end_id
18319338,1.002938,550,413,0,42.291749,-83.682617,13.421875,0
18319468,1.002938,550,413,95300,42.309110,-83.677268,27.218750,1
18315381,1.015493,540,483,0,42.291189,-83.799998,19.000000,2
18315925,1.015493,540,483,478700,42.281186,-83.747657,17.000000,3
18258980,1.017633,156,912,0,42.309303,-83.677652,9.000000,1
18259669,1.017633,156,912,597300,42.286800,-83.780572,58.000000,-1
18323652,1.025782,588,73,0,42.309431,-83.677319,0.000000,1
18324801,1.025782,588,73,1033500,42.244419,-83.764921,16.000000,4
18280024,1.054483,267,647,0,42.289890,-83.737930,35.000000,5
18280518,1.054483,267,647,469700,42.308822,-83.678517,73.000000,1


Now, we fill in the identifiers into the appropriate columns in the trip DataFrame.

In [18]:
trip_df['NodeIni'] = trip_df.apply(get_trip_ini, axis=1)
trip_df['NodeEnd'] = trip_df.apply(get_trip_end, axis=1)

In [19]:
trip_df.head(10)

,DayNum,VehId,Count,NodeIni,NodeEnd
0,1.002938,550,131,0,1
1,1.015493,540,545,2,3
2,1.017633,156,690,1,-1
3,1.025782,588,1150,1,4
4,1.054483,267,495,5,1
5,1.058991,11,396,6,7
6,1.062756,130,452,8,9
7,1.065486,174,273,10,11
8,1.082547,374,471,12,3
9,1.101627,156,697,13,14


In [20]:
df0 = ends_df[ends_df.end_id == 5]

In [21]:
s = df0['Latitude[deg]'].min()
n = df0['Latitude[deg]'].max()
w = df0['Longitude[deg]'].min()
e = df0['Longitude[deg]'].max()

In [22]:
(s, w, n, e)

(42.2892022222, -83.7408405556, 42.2912408333, -83.7378394444)

In [23]:
import requests
import json

In [24]:
overpass_url = "http://overpass-api.de/api/interpreter"

In [25]:
overpass_query = "[out:json];(way[highway]({0},{1},{2},{3}););(._;>;);out body;".format(s, w, n, e)

In [26]:
overpass_query

'[out:json];(way[highway](42.2892022222,-83.7408405556,42.2912408333,-83.7378394444););(._;>;);out body;'

In [27]:
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [28]:
list(data.keys())

['version', 'generator', 'osm3s', 'elements']

In [29]:
# data['elements']

In [30]:
from osm.models import OSMNode, OSMWay

In [31]:
nodes = [OSMNode(n['id'], n['lat'], n['lon']) for n in data['elements'] if n['type'] == 'node']

In [32]:
ways = [OSMWay(w['id'], w['nodes'], w['tags']) for w in data['elements'] if w['type'] == 'way']

In [33]:
nodes

[OSMNode(nid=62486495, lat=42.2902074, lon=-83.7409516,
 OSMNode(nid=62486497, lat=42.2903016, lon=-83.7409793,
 OSMNode(nid=62486499, lat=42.2903844, lon=-83.7409465,
 OSMNode(nid=62486501, lat=42.2909053, lon=-83.7407497,
 OSMNode(nid=62486503, lat=42.2911184, lon=-83.7406634,
 OSMNode(nid=62486505, lat=42.2930011, lon=-83.739937,
 OSMNode(nid=62486507, lat=42.2940514, lon=-83.7395408,
 OSMNode(nid=62490557, lat=42.2973528, lon=-83.736998,
 OSMNode(nid=62494758, lat=42.2899286, lon=-83.7405761,
 OSMNode(nid=62494762, lat=42.28903, lon=-83.7395313,
 OSMNode(nid=62499610, lat=42.2893576, lon=-83.7388934,
 OSMNode(nid=62499611, lat=42.2891316, lon=-83.7383457,
 OSMNode(nid=62501564, lat=42.2895035, lon=-83.7386314,
 OSMNode(nid=62501566, lat=42.2895997, lon=-83.7385239,
 OSMNode(nid=62501568, lat=42.2896973, lon=-83.7384449,
 OSMNode(nid=62501571, lat=42.2898006, lon=-83.7383804,
 OSMNode(nid=62501582, lat=42.2903049, lon=-83.7372716,
 OSMNode(nid=62501598, lat=42.2906971, lon=-83.73785

In [34]:
ways

[OSMWay(wid=8722317, name=Wright Street),
 OSMWay(wid=8723036, name=Swift Street),
 OSMWay(wid=8723579, name=Broadway Street),
 OSMWay(wid=8733979, name=Plymouth Road),
 OSMWay(wid=8734192, name=Pontiac Trail),
 OSMWay(wid=8734271, name=Long Shore Drive),
 OSMWay(wid=60923483),
 OSMWay(wid=61508808, name=Moore Street),
 OSMWay(wid=160964165, name=Swift Street),
 OSMWay(wid=222784526, name=Broadway Street),
 OSMWay(wid=222787431, name=Broadway Street),
 OSMWay(wid=225434709),
 OSMWay(wid=314284475),
 OSMWay(wid=401624615),
 OSMWay(wid=410015012, name=Maiden Lane),
 OSMWay(wid=410027434, name=Broadway Street),
 OSMWay(wid=410027435, name=Wall Street),
 OSMWay(wid=412071382),
 OSMWay(wid=413189666, name=Pontiac Trail),
 OSMWay(wid=437517025, name=Argo Dam B2B),
 OSMWay(wid=441448135, name=Argo Dam B2B),
 OSMWay(wid=441448139, name=Broadway Street),
 OSMWay(wid=441448162, name=Moore Street),
 OSMWay(wid=441448163, name=Moore Street),
 OSMWay(wid=441566047, name=Broadway Street),
 OSMWay(wi

In [ ]:
import math

In [ ]:
math.log2(631997299)

In [ ]:
locator = Nominatim(user_agent='ann_arbor')

In [ ]:
coordinates = (42.292475, -83.682453)
location = locator.reverse(query=coordinates, timeout=60)
location.raw

## Building the Trip Graph

By looking at the data in the `trip_df` DataFrame, we can immediately see an interesting analysis opportunity. Between two nodes (`NodeIni` and `NodeEnd`), we can build an edge. Yes, I am talking about a directed graph. Let's build it using the known edge information (filtering out any edge with a _noise_ endpoint), and use their frequency as the edge weight.

In [ ]:
ids = ends_df.end_id.to_numpy()

In [ ]:
id_pairs = np.reshape(ids, (ids.shape[0] // 2, 2))

In [ ]:
id_pairs

We can now build the raw list of edges.

In [ ]:
edges = [(id_pairs[i,0], id_pairs[i,1]) for i in range(id_pairs.shape[0]) if id_pairs[i,0] != -1 and id_pairs[i,1] != -1]

A `Counter` object helps in the edge weight calculation and, in a way, embodies the first representation of the trip graph.

In [ ]:
cnt = Counter(edges)

In [ ]:
len(cnt)

## Optional - Create a Graph from Trip Data

For illustration purposes, we now convert the edge informatio into a directed graph. We use the `networkx` package to create the graph and then save it into a `gephi` compatible file format.

In [ ]:
dg = nx.DiGraph()

Create a list of tuples where each tuple contains the terminal node identifiers as well as the trip count.

In [ ]:
data = [(p[0], p[1], cnt[p]) for p in cnt]

This list is now directly fed into the directed graph to instantiate it.

In [ ]:
dg.add_weighted_edges_from(data)

Save the graph to a `gexf` file. This file format is recognized by the `gephi` graph software.

In [ ]:
nx.write_gexf(dg, "./data/ved.gexf")

In [ ]:
edge_df = pd.DataFrame(data, columns=['NodeIni', 'NodeEnd', 'Count'])

In [ ]:
edge_df = edge_df.sort_values(by=['Count'], ascending=False)

In [ ]:
edge_df.head(10)

In [ ]:
m = make_link_map(node_ini=25, node_end=26)

In [ ]:
m.save("./html/map-25-26.html")